In [1]:
import pandas as pd
import numpy as np
import geopandas as gp
import matplotlib.pylab as pl
from fiona.crs import from_epsg
import fiona
from matplotlib import colors
import shapely
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap    
%matplotlib inline
pd.set_option('display.height',1000)
pd.set_option('display.max_row',500)
pd.set_option('display.max_columns',500)
pd.set_option("display.width", 1000)

height has been deprecated.



### Objective
The objective of this notebook is to solve the Overlap problem present in some block-groups. By CADC suggestion, the Water District that should be assigned when overlapping is present, it’s the one with smaller area. 
This is based on the nature of water districts, some bigger water districts that contain smaller ones are created for water wholesale, while smaller ones are the ones that directly serve the costumer.


In [2]:
#load the file form the previews notebook
bg10 = gp.GeoDataFrame.from_file('../Data/GeoData/BG2010/tl_2010_06_bg10_Complete.geojson')
print bg10.shape
bg10.head(1)

(23212, 16)


,ALAND10,AWATER10,BLKGRPCE10,COUNTYFP10,FUNCSTAT10,GEOID10,INTPTLAT10,INTPTLON10,MTFCC10,NAMELSAD10,PWSID,PWSID_str,STATEFP10,TRACTCE10,geometry,id
0,1570668,0,3,053,S,060530107013,+36.6007541,-121.7026925,G5030,Block Group 3,None,None,06,010701,"POLYGON ((-121.695295 36.611978, -121.695835 3...",0


In [3]:
#load file provided by CADC, list of water districts with their polygon
wa_d = gp.GeoDataFrame.from_file('../Data/Supplier/suppliers.geojson')
print wa_d.shape
wa_d.head(1)

(410, 16)


,cartodb_id,city,county,detailed_c,displayed,general_ch,geom_3310,geometry,gid,parcels_in,pwsid,reference,sbx7_7_tar,supplier_n,to_be_proc,utility_id
0,219,,,,1,N/A,0106000020EE0C00000100000001030000000100000069...,"(POLYGON ((-122.000388 37.34519, -122.000371 3...",78,0,CA4310001,,1,California Water Service Company Los Altos/Sub...,0,227


In [4]:
print bg10[bg10.PWSID=='Overlap'].shape
bg10[bg10.PWSID=='Overlap'].head(3)

(1364, 16)


,ALAND10,AWATER10,BLKGRPCE10,COUNTYFP10,FUNCSTAT10,GEOID10,INTPTLAT10,INTPTLON10,MTFCC10,NAMELSAD10,PWSID,PWSID_str,STATEFP10,TRACTCE10,geometry,id
180,614499,0,3,083,S,060830001023,+34.4409365,-119.7568967,G5030,Block Group 3,Overlap,"CA4210004, CA4210010",06,000102,"POLYGON ((-119.759786 34.440687, -119.75978 34...",180
190,892404,0,7,083,S,060830020077,+34.9036946,-120.4204500,G5030,Block Group 7,Overlap,"CA4210011, CA4210016",06,002007,"POLYGON ((-120.418023 34.896043, -120.418036 3...",190
335,2595170,106198,2,083,S,060830013062,+34.4120864,-119.7495788,G5030,Block Group 2,Overlap,"CA4210004, CA4210010",06,001306,"POLYGON ((-119.756013 34.417628, -119.756012 3...",335


### Methodology
The following code compares the area from water districts that overlap and assigns the one with the smaller area to the block-group. The column PWSID_str contains the list of WD that overlap, it is splited and then the area is compared selecting the smaller as the right WD.

In [5]:

for block in bg10[bg10.PWSID=='Overlap'].index:
    smaller_name = np.nan
    smaller_area = wa_d['geometry'].area.max()
    #print bg10.loc[block,'PWSID_str'].split(', ')
    for wa in bg10.loc[block,'PWSID_str'].split(', '):
        
        if smaller_area >= wa_d.loc[wa_d.loc[wa_d.pwsid == wa].index[0],'geometry'].area:
            smaller_area = wa_d.loc[wa_d.loc[wa_d.pwsid == wa].index[0],'geometry'].area
            smaller_name = wa
    #print smaller_name
    bg10.loc[block, 'PWSID'] = smaller_name
        

In [6]:
print bg10[bg10.PWSID=='Overlap'].shape
#bg10[bg10.PWSID=='Overlap'].head()

(0, 16)


In [7]:
#The result is saved as tl_2010_06_bg10_Complete_Solved_Overlaps.geojson
with open('../Data/GeoData/BG2010/tl_2010_06_bg10_Complete_Solved_Overlaps.geojson', 'w') as f:
    f.write(bg10.to_json())